Standard imports

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import datetime

Text cleaner imports

In [283]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
import re

stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ysgard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


This imports the hedonometer words. Could be used for sentiment analysis.

In [178]:
hedonometer = pd.read_csv("../Hedonometer.csv")

#initialize the categorization
negative_words, neutral_words, positive_words, all_words = {}, {}, {}, {}

# how clearly valent do we want the words?
tolerance = 0

# demark words that are clearly negative or positive (neutral is within tolerance standard deviations of 5)
for i in hedonometer.index:
    score = (hedonometer['Happiness Score'][i]-5)/5
    if score+tolerance*hedonometer['Standard Deviation of Ratings'][i]/5<0:
        negative_words[hedonometer.Word[i]] = score
    elif score-tolerance*hedonometer['Standard Deviation of Ratings'][i]/5>5:
        positive_words[hedonometer.Word[i]] = score
    else: 
        neutral_words[hedonometer.Word[i]] = score
    all_words[hedonometer.Word[i]] = score

This can save stuff for later usage, in case they're handy.

In [30]:
def tweet_saver(tweets, file='TWagner.csv'):
    with open(file, 'a', encoding='utf8') as saveTweets:
        for tweet in tweets: 
            saveTweets.write(str(tweet._json)+'\n')
    
def user_saver(user, file='UUagner.csv'):
    with open(file, 'a', encoding='utf8') as saveUser:
        saveUser.write(str(user)+'\n')

Personal info please don't use.

In [2]:
ck = 'QxPH3Ud4nFeHxfiOsnzNr2sql'
cs = ''
at = '1393253063528431621-Tk8YAgtU3B4SwJa4DmHlhfGlcaiWN6'
ats = ''

This creates an instance of the API so that I can pull Tweets from Twitter.

In [3]:
# create OAuthHandler object
authenticator = tweepy.OAuthHandler(ck, cs)
# set access token and secret
authenticator.set_access_token(at, ats)
# create tweepy API object to fetch tweets
API = tweepy.API(authenticator, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Practicing/experimenting with followers.

In [203]:
API.show_friendship(source_screen_name='DougEpisodes', target_id=1015024528337129473)

(Friendship(_api=<tweepy.api.API object at 0x000001F975D9C588>, _json={'id': 2485562569, 'id_str': '2485562569', 'screen_name': 'DougEpisodes', 'following': False, 'followed_by': True, 'live_following': False, 'following_received': None, 'following_requested': None, 'notifications_enabled': None, 'can_dm': True, 'blocking': None, 'blocked_by': None, 'muting': None, 'want_retweets': None, 'all_replies': None, 'marked_spam': None}, id=2485562569, id_str='2485562569', screen_name='DougEpisodes', following=False, followed_by=True, live_following=False, following_received=None, following_requested=None, notifications_enabled=None, can_dm=True, blocking=None, blocked_by=None, muting=None, want_retweets=None, all_replies=None, marked_spam=None),
 Friendship(_api=<tweepy.api.API object at 0x000001F975D9C588>, _json={'id': 1015024528337129473, 'id_str': '1015024528337129473', 'screen_name': 'newrockytweets', 'following': True, 'followed_by': False, 'following_received': None, 'following_request

Let's make a dataframe of activity

In [206]:
def get_activity(user, count=20, local=False):
    # Make sure we're finding the right user
    if local:
        with open(user, 'r', encoding='utf8') as reader:
            pass
    elif type(user)==str:
        apitivity = API.user_timeline(screen_name=user, count=count)
    elif type(user)==int:
        apitivity = API.user_timeline(user_id=user, count=count)
    else:
        raise Exception("Bad user.") 
        
    # Make a list of the 
    activiray = []
    for tweet in apitivity:
        try: 
            tweet.retweeted_status
            is_retweet = True
        except: 
            is_retweet = False
        activiray.append([tweet.text,
                         tweet.id,
                         is_retweet,
                         tweet.is_quote_status,
                         type(tweet.in_reply_to_status_id)==int,
                         tweet.in_reply_to_status_id,
                         tweet.in_reply_to_user_id,
                         tweet.retweet_count,
                         tweet.favorite_count,
                         tweet.created_at])
        
    # turn it into a dataframe
    return pd.DataFrame(activiray, columns=['Text', 'Id', 'Is_retweet', 'Is_quote', 'Is_reply', 'Reply_of', 'Replying_to', 'Retweets', 'Favorites', 'Created'])

In [281]:
def prep_activity(activity, count=20, 
                   streak_lengths=[datetime.timedelta(hours=1),datetime.timedelta(days=1)], 
                   per_periods=[datetime.timedelta(hours=1),datetime.timedelta(days=1)]):
    # Did we already get the activity?
    if type(activity)!=pd.core.frame.DataFrame:
        activity = get_activity(activity, count=count)
    
    #clean the text
    cleets = []
    for twittre in activity.Text:
        #this removes mentions
        twittre = re.sub(r'@[A-Za-z0-9_]+', '', twittre)
        #this removes urls
        twittre = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', twittre)        
        #this removes everything but letters from the str
        twittre = re.sub('[^a-zA-Z]',' ',twittre)
        #this makes everything lowercase, then turns everything that is separated by a space into its own str
        twittre = twittre.lower().split()
        #this removes stop words and stems everything else
        twittre = [stemmer.stem(word) for word in twittre if (word not in stop_words)]
        # this puts it all back together with spaces inbetween
        twittre = ' '.join(twittre)
        cleets.append(twittre)
    activity['Cleaned'] = cleets
    
    # record the time since the last status update
    tsl = [activity.Created[n].to_pydatetime() - activity.Created[n+1].to_pydatetime()
                                   for n in range(len(activity)-1)] + [np.nan]
    activity['Time_since_last'] = tsl.copy()
    
    #record how many times in a row the last activity was less than length 
    for length in streak_lengths:
        #record when it was less than length
        streak_break = [time>length for time in tsl[:-1]]+[True]
        
        #intialize beginning of a streak
        start = 0
        streak = []
        for end in range(len(streak_break)):
            #if this is the end of a streak,  
            if streak_break[end]:
                #record it,
                streak += [end+1-start for indx in range(end+1-start)]
                #and begin the next
                start = end+1
        
        #finally, add it to the data frame
        activity['Streak of '+str(length)] = streak
        
    #record how much activity occured within a given period 
    for period in per_periods:
        per = []
        for n in range(len(tsl)):
            elapsed = tsl[n]
            count, m = 0, 1
            #first we look at earlier tweets
            while (m+n<len(tsl)) and (elapsed<=period/2):
                count += 1
                try: elapsed += tsl[m+n]
                except: pass
                m += 1
            m = 0
            elapsed = datetime.timedelta(0)
            #then later tweets
            while n>=m and elapsed<=period/2:
                count += 1
                m += 1
                try: elapsed += tsl[n-m]
                except: pass
            per += [count]
        activity['Per '+str(period)] = per

In [136]:
funtime = relativedelta(activity.Created[0], activity.Created[1])

In [8]:
with open('./erdos_core/0.txt', 'r', encoding='utf8') as reader:
    tweets = reader.readlines()

In [282]:
activity = get_activity('erdosinstitute')
prep_activity(activity)
activity

,Text,Id,Is_retweet,Is_quote,Is_reply,Reply_of,Replying_to,Retweets,Favorites,Created,Cleaned,Time_since_last,Streak of 1:00:00,"Streak of 1 day, 0:00:00",Per 1:00:00,"Per 1 day, 0:00:00"
0,RT @AstronomerPat: @RacineSwick @WorkWithVari ...,1395210702869118982,True,False,False,NaN,NaN,1,0,2021-05-20 02:52:07,rt one week data scienc lectur,2 days 12:09:26,1,1,1,1
1,Dreams can come true @AstronomerPat https://t....,1394302356267880449,False,True,False,NaN,NaN,0,1,2021-05-17 14:42:41,dream come true,0 days 00:00:38,2,2,2,2
2,@AstronomerPat @WorkWithVari Same excellent co...,1394302197006053379,False,False,True,1.394291e+18,9.130586e+17,0,1,2021-05-17 14:42:03,excel content screen,10 days 14:15:05,2,2,2,2
3,Our May Data Science Boot Camp is in full swin...,1390463130627674113,False,True,False,NaN,NaN,0,4,2021-05-07 00:26:58,may data scienc boot camp full swing photo credit,0 days 00:09:20,2,2,2,2
4,@AstronomerPat @MattOsborne71: This is the mos...,1390460782647906306,False,False,True,1.390296e+18,9.130586e+17,1,6,2021-05-07 00:17:38,retweet ever photo data scienc boot camp,6 days 12:42:47,2,2,2,2
5,RT @columbusbiz1st: How an OSU math professor ...,1388094491609997320,True,False,False,NaN,NaN,2,0,2021-04-30 11:34:51,rt osu math professor match ph privat sector job,1 days 10:16:08,1,1,1,1
6,Our May Data Science Boot Camp with over 350 g...,1387577050735812608,False,False,False,NaN,NaN,1,9,2021-04-29 01:18:43,may data scienc boot camp grad student postdoc...,6 days 09:23:46,1,1,1,1
7,Invitations to Industry continues tonight with...,1385260846515773444,False,False,False,NaN,NaN,1,1,2021-04-22 15:54:57,invit industri continu tonight kenneth maynard...,1 days 16:01:56,1,1,1,1
8,RT @OSUmath: The Hidden Figures course develop...,1384656382406840329,True,False,False,NaN,NaN,3,0,2021-04-20 23:53:01,rt hidden figur cours develop dr ranthoni edmo...,9 days 03:36:50,1,1,1,1
9,Invitations to Industry continues Monday 04/12...,1381340320051384321,False,False,False,NaN,NaN,0,0,2021-04-11 20:16:11,invit industri continu monday thoma sznigir st...,8 days 19:50:50,1,1,1,1


In [208]:
clean_activity(activity)

In [209]:
activity

,Text,Id,Created,Is reply,Reply_of,Replying_to,Retweets,Favorites,Is_retweet,Time_since_last,Streak of 1:00:00,"Per 1 day, 0:00:00"
0,RT @AstronomerPat: @RacineSwick @WorkWithVari ...,1395210702869118982,2021-05-20 02:52:07,False,NaN,NaN,1,0,True,2 days 12:09:26,1,1
1,Dreams can come true @AstronomerPat https://t....,1394302356267880449,2021-05-17 14:42:41,False,NaN,NaN,0,1,False,0 days 00:00:38,2,2
2,@AstronomerPat @WorkWithVari Same excellent co...,1394302197006053379,2021-05-17 14:42:03,True,1.394291e+18,9.130586e+17,0,1,False,10 days 14:15:05,2,2
3,Our May Data Science Boot Camp is in full swin...,1390463130627674113,2021-05-07 00:26:58,False,NaN,NaN,0,4,False,0 days 00:09:20,2,2
4,@AstronomerPat @MattOsborne71: This is the mos...,1390460782647906306,2021-05-07 00:17:38,True,1.390296e+18,9.130586e+17,1,6,False,6 days 12:42:47,2,2
5,RT @columbusbiz1st: How an OSU math professor ...,1388094491609997320,2021-04-30 11:34:51,False,NaN,NaN,2,0,True,1 days 10:16:08,1,1
6,Our May Data Science Boot Camp with over 350 g...,1387577050735812608,2021-04-29 01:18:43,False,NaN,NaN,1,9,False,6 days 09:23:46,1,1
7,Invitations to Industry continues tonight with...,1385260846515773444,2021-04-22 15:54:57,False,NaN,NaN,1,1,False,1 days 16:01:56,1,1
8,RT @OSUmath: The Hidden Figures course develop...,1384656382406840329,2021-04-20 23:53:01,False,NaN,NaN,3,0,True,9 days 03:36:50,1,1
9,Invitations to Industry continues Monday 04/12...,1381340320051384321,2021-04-11 20:16:11,False,NaN,NaN,0,0,False,8 days 19:50:50,1,1


In [5]:
def build_core(user, breadth=10, depth=2, core=[], save=False):
    #type user as a screen name so this works
    howrite = 'a'
    
    # Make sure we're finding the right user
    if type(user)==str:
        user = API.get_user(screen_name=user).id
        howrite = 'w'
    elif type(user)!=int:
        raise Exception("Bad user.")
    
    #make sure we have good data
    try:
        friends = API.friends_ids(user_id=user)
        friends[0]
    except: 
        return False
    
    #one list for user's mutuals, one list for everyone
    mutuals = []
    core.append(user)
    if save:
        try:
            with open(save, howrite) as save_file:
                save_file.write(str(user)+'\n')
        except: 
            raise Exception('Bad save file')
    
    #we won't delve too deep or too greedily
    if depth:
        #find random friends of user who follow user back (mutuals)
        for friend in np.random.permutation(friends):
            friend = int(friend)
            if friend not in core:
                if API.show_friendship(source_id=friend, target_id=user)[0].following:
                    #find their mutuals, but only add them to the list if it's good data
                    if build_core(friend, breadth=breadth, depth=depth-1, core=core, save=save):
                        #add them to the list
                        mutuals.append(friend)
                    #but only some of them
                    if len(mutuals)==breadth:
                        print(friend,'has mutuals',mutuals)
                        break
    
    #and here's the list
    return core

In [6]:
def build_crust(core, breadth=100, crust=[], save=False):
    #initialize a list for the crust
    crust = core.copy()
    
    #and a save file if neccessary
    if save:
        try:
            with open(save, 'w') as save_file:
                for core_member in core:
                    save_file.write(str(core_member)+'\n')
        except: 
            raise Exception('Bad save file')
        
    #for each member of the core...
    for core_member in core.copy():   
        skip_number = 0
        #add breadth number...
        follow_number = 0
        #of the core  member's...
        followers = np.random.permutation(API.followers_ids(user_id=core_member))
        
        #followers to the crust...
        for follower in followers:
            follower = int(follower)
            
            #if they are accessible...
            try:
                getted = API.get_user(user_id=follower)
                if getted.protected:
                    raise Exception('Skip this one')
            except:
                skip_number += 1
                continue
                
            #and aren't already in it
            if follower not in crust:
                crust.append(follower)
                if save:
                    with open(save, 'a') as save_file:
                        save_file.write(str(follower)+'\n')
                follow_number += 1
                if follow_number == breadth:
                    break
            else:
                skip_number += 1
        #keep tabs on how the script is going
        print('got',follow_number,'followers of',core_member,'while skipping',skip_number)
    
    return crust

In [49]:
def frame_data(core, crust, data=[], save=False):
    #we assign core members as features
    columns = ['core_'+str(n) for n in range(len(core))]
    if save: 
        with open(save, 'r') as save_file:
            text = [''] + save_file.readlines()
        for column in columns:
            text[0] += ',' + str(column)
        text[0] += '\n'
        with open(save, 'w') as save_file:
            save_file.writelines(text)
        
    #now we see which crust members are following which core members
    for n, crust_member in enumerate(crust):
        #this will hold what we find
        next_line = []
        #look up who the crust member is following
        following, pages = API.friends_ids(user_id=crust_member, cursor=-1)
        
        for core_member in core:
            #page through if necessary
            while pages[1] and (core_member not in following):
                more_following, pages = API.friends_ids(user_id=crust_member, cursor=pages[1])
                following += more_following
            next_line.append(core_member in following)
        
        #We'll say people follow themselves
        if n < len(core):
            next_line[n]==True
        
        #now we store it
        data.append(next_line)
        if save: 
            with open(save, 'r') as save_file:
                text = save_file.readlines()
            text[n+1] = text[n+1][:-1]
            for datum in next_line:
                text[n+1] += ',' + str(datum)
            text[n+1] += '\n'
            with open(save, 'w') as save_file:
                save_file.writelines(text)
        print('Stored data for member', n+1, 'of', len(crust))
    
    #and finally put it all together
    return pd.DataFrame(data, index=crust, columns=columns)

In [5]:
toast_frame = pd.read_csv('erdos_full_core.csv')
toast_frame.index = toast_frame['index']
toast_frame = toast_frame[['core_'+str(n) for n in range(105)]]
toast_frame

,core_0,core_1,core_2,core_3,core_4,core_5,core_6,core_7,core_8,core_9,...,core_95,core_96,core_97,core_98,core_99,core_100,core_101,core_102,core_103,core_104
index,,,,,,,,,,,,,,,,,,,,,
840727059001417728,True,True,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
773706709,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,True,False,False
818967308966051840,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
135516328,False,True,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
32272194,False,True,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013701563108462592,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1102503628433170432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
1908769994,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [ ]:

for i, crusty in enumerate(crust):
    tweets = API.user_timeline(user_id=crusty, count=100)
    tweets_array.append(tweets)
    tweet_saver(tweets, '../Messing around/erdos_crust/'+str(i)+'.txt')
    print('Found',len(tweets),'from number', i)

Found 98 from number 0
Found 100 from number 1
Found 99 from number 2
Found 61 from number 3
Found 64 from number 4
Found 100 from number 5
Found 6 from number 6
Found 100 from number 7
Found 8 from number 8
Found 100 from number 9
Found 3 from number 10
Found 0 from number 11
Found 99 from number 12
Found 100 from number 13
Found 100 from number 14
Found 100 from number 15
Found 100 from number 16
Found 99 from number 17
Found 100 from number 18
Found 42 from number 19
Found 100 from number 20
Found 100 from number 21
Found 100 from number 22
Found 36 from number 23
Found 100 from number 24
Found 100 from number 25
Found 100 from number 26
Found 63 from number 27
Found 100 from number 28
Found 100 from number 29
Found 0 from number 30
Found 99 from number 31
Found 99 from number 32
Found 98 from number 33
Found 100 from number 34
Found 100 from number 35
Found 16 from number 36
Found 2 from number 37
Found 68 from number 38
Found 99 from number 39
Found 100 from number 40
Found 100 fr

Found 100 from number 327
Found 2 from number 328
Found 100 from number 329
Found 54 from number 330
Found 100 from number 331
Found 100 from number 332
Found 100 from number 333
Found 100 from number 334
Found 16 from number 335
Found 100 from number 336
Found 8 from number 337
Found 97 from number 338
Found 7 from number 339
Found 100 from number 340
Found 100 from number 341
Found 99 from number 342
Found 100 from number 343
Found 97 from number 344
Found 100 from number 345
Found 100 from number 346
Found 100 from number 347
Found 100 from number 348
Found 100 from number 349
Found 100 from number 350
Found 100 from number 351
Found 100 from number 352
Found 1 from number 353
Found 100 from number 354
Found 100 from number 355
Found 100 from number 356
Found 36 from number 357
Found 100 from number 358
Found 100 from number 359
Found 100 from number 360
Found 1 from number 361
Found 100 from number 362
Found 100 from number 363
Found 100 from number 364
Found 100 from number 365
Fo

Found 98 from number 650
Found 100 from number 651
Found 100 from number 652
Found 99 from number 653
Found 100 from number 654
Found 99 from number 655
Found 100 from number 656
Found 100 from number 657
Found 100 from number 658
Found 100 from number 659
Found 100 from number 660
Found 100 from number 661
Found 100 from number 662
Found 100 from number 663
Found 100 from number 664
Found 81 from number 665
Found 99 from number 666
Found 100 from number 667
Found 100 from number 668
Found 100 from number 669
Found 100 from number 670
Found 100 from number 671
Found 7 from number 672
Found 100 from number 673
Found 20 from number 674
Found 100 from number 675
Found 0 from number 676
Found 100 from number 677
Found 100 from number 678
Found 100 from number 679
Found 97 from number 680
Found 100 from number 681
Found 100 from number 682
Found 100 from number 683
Found 100 from number 684
Found 100 from number 685
Found 100 from number 686
Found 100 from number 687
Found 100 from number 6

In [5]:
2

2

In [75]:
with open('erdos_full_sample.csv', 'r') as reader:
    lines = reader.readlines()

In [81]:
crust += [int(line) for line in lines[154:]]

In [66]:
API.get_user(screen_name='UnprotectedMe')

User(_api=<tweepy.api.API object at 0x000002327876BF48>, _json={'id': 80076948, 'id_str': '80076948', 'name': 'khadija', 'screen_name': 'unprotectedme', 'location': '', 'profile_location': None, 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': True, 'followers_count': 2, 'friends_count': 63, 'listed_count': 0, 'created_at': 'Mon Oct 05 17:24:55 +0000 2009', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 5, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '1A1B1F', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme9/bg.gif', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme9/bg.gif', 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https':

Testing NLTK

In [256]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
import re

stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ysgard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'rt astronomerpat racineswick workwithvari one week erdosinstitut data scienc lectur'

In [262]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [247]:
json_toast = '''_json={\'created_at\': \'Thu May 20 02:52:07 +0000 2021\', \'id\': 1395210702869118982, \'id_str\': \'1395210702869118982\', \'text\': \'RT @AstronomerPat: @RacineSwick @WorkWithVari This was during one of this week’s @erdosinstitute data science lectures!\', \'truncated\': False, \'entities\': {\'hashtags\': [], \'symbols\': [], \'user_mentions\': [{\'screen_name\': \'AstronomerPat\', \'name\': \'Patrick Vallely 🛰️🌌🔭\', \'id\': 913058608883142657, \'id_str\': \'913058608883142657\', \'indices\': [3, 17]}, {\'screen_name\': \'RacineSwick\', \'name\': \'Racine Swick\', \'id\': 1231793881672486912, \'id_str\': \'1231793881672486912\', \'indices\': [19, 31]}, {\'screen_name\': \'WorkWithVari\', \'name\': \'Vari®\', \'id\': 1266638119, \'id_str\': \'1266638119\', \'indices\': [32, 45]}, {\'screen_name\': \'erdosinstitute\', \'name\': \'The Erdős Institute\', \'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'indices\': [81, 96]}], \'urls\': []}, \'source\': \'<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>\', \'in_reply_to_status_id\': None, \'in_reply_to_status_id_str\': None, \'in_reply_to_user_id\': None, \'in_reply_to_user_id_str\': None, \'in_reply_to_screen_name\': None, \'user\': {\'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'name\': \'The Erdős Institute\', \'screen_name\': \'erdosinstitute\', \'location\': \'United States\', \'description\': \'Helping PhDs get jobs they love. \\nFollow us on LinkedIn: https://t.co/pNGRyjotVd\', \'url\': \'https://t.co/MoNoEsytrh\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/MoNoEsytrh\', \'expanded_url\': \'http://erdosinstitute.org\', \'display_url\': \'erdosinstitute.org\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': [{\'url\': \'https://t.co/pNGRyjotVd\', \'expanded_url\': \'https://www.linkedin.com/company/the-erdős-institute\', \'display_url\': \'linkedin.com/company/the-er…\', \'indices\': [57, 80]}]}}, \'protected\': False, \'followers_count\': 304, \'friends_count\': 228, \'listed_count\': 4, \'created_at\': \'Sun Mar 12 00:52:05 +0000 2017\', \'favourites_count\': 708, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 516, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/840727059001417728/1610744135\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': False, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, \'geo\': None, \'coordinates\': None, \'place\': None, \'contributors\': None, \'retweeted_status\': {\'created_at\': \'Mon May 17 14:56:49 +0000 2021\', \'id\': 1394305912622489602, \'id_str\': \'1394305912622489602\', \'text\': \'@RacineSwick @WorkWithVari This was during one of this week’s @erdosinstitute data science lectures!\', \'truncated\': False, \'entities\': {\'hashtags\': [], \'symbols\': [], \'user_mentions\': [{\'screen_name\': \'RacineSwick\', \'name\': \'Racine Swick\', \'id\': 1231793881672486912, \'id_str\': \'1231793881672486912\', \'indices\': [0, 12]}, {\'screen_name\': \'WorkWithVari\', \'name\': \'Vari®\', \'id\': 1266638119, \'id_str\': \'1266638119\', \'indices\': [13, 26]}, {\'screen_name\': \'erdosinstitute\', \'name\': \'The Erdős Institute\', \'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'indices\': [62, 77]}], \'urls\': []}, \'source\': \'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>\', \'in_reply_to_status_id\': 1394294574806278144, \'in_reply_to_status_id_str\': \'1394294574806278144\', \'in_reply_to_user_id\': 1231793881672486912, \'in_reply_to_user_id_str\': \'1231793881672486912\', \'in_reply_to_screen_name\': \'RacineSwick\', \'user\': {\'id\': 913058608883142657, \'id_str\': \'913058608883142657\', \'name\': \'Patrick Vallely 🛰️🌌🔭\', \'screen_name\': \'AstronomerPat\', \'location\': \'NV➡OK➡OH\', \'description\': \'Studying supernovae with ASAS-SN & TESS | @NSFGRFP fellow at The™ @OhioState | @UofOklahoma Class of 2017 | Amateur photographer and big cat enthusiast | He/Him\', \'url\': \'https://t.co/AEt85T0LIF\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/AEt85T0LIF\', \'expanded_url\': \'https://u.osu.edu/vallely.7/\', \'display_url\': \'u.osu.edu/vallely.7/\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': []}}, \'protected\': False, \'followers_count\': 1262, \'friends_count\': 664, \'listed_count\': 12, \'created_at\': \'Wed Sep 27 15:11:50 +0000 2017\', \'favourites_count\': 39911, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 6678, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/913058608883142657/1540879883\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': True, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, \'geo\': None, \'coordinates\': None, \'place\': None, \'contributors\': None, \'is_quote_status\': False, \'retweet_count\': 1, \'favorite_count\': 3, \'favorited\': False, \'retweeted\': False, \'lang\': \'en\'}, \'is_quote_status\': False, \'retweet_count\': 1, \'favorite_count\': 0, \'favorited\': False, \'retweeted\': False, \'lang\': \'en\'}, created_at=datetime.datetime(2021, 5, 20, 2, 52, 7), id=1395210702869118982, id_str=\'1395210702869118982\', text=\'RT @AstronomerPat: @RacineSwick @WorkWithVari This was during one of this week’s @erdosinstitute data science lectures!\', truncated=False, entities={\'hashtags\': [], \'symbols\': [], \'user_mentions\': [{\'screen_name\': \'AstronomerPat\', \'name\': \'Patrick Vallely 🛰️🌌🔭\', \'id\': 913058608883142657, \'id_str\': \'913058608883142657\', \'indices\': [3, 17]}, {\'screen_name\': \'RacineSwick\', \'name\': \'Racine Swick\', \'id\': 1231793881672486912, \'id_str\': \'1231793881672486912\', \'indices\': [19, 31]}, {\'screen_name\': \'WorkWithVari\', \'name\': \'Vari®\', \'id\': 1266638119, \'id_str\': \'1266638119\', \'indices\': [32, 45]}, {\'screen_name\': \'erdosinstitute\', \'name\': \'The Erdős Institute\', \'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'indices\': [81, 96]}], \'urls\': []}, source=\'Twitter Web App\', source_url=\'https://mobile.twitter.com\', in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=None, in_reply_to_user_id_str=None, in_reply_to_screen_name=None, author=User(_api=<tweepy.api.API object at 0x00000208D4918948>, _json={\'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'name\': \'The Erdős Institute\', \'screen_name\': \'erdosinstitute\', \'location\': \'United States\', \'description\': \'Helping PhDs get jobs they love. \\nFollow us on LinkedIn: https://t.co/pNGRyjotVd\', \'url\': \'https://t.co/MoNoEsytrh\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/MoNoEsytrh\', \'expanded_url\': \'http://erdosinstitute.org\', \'display_url\': \'erdosinstitute.org\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': [{\'url\': \'https://t.co/pNGRyjotVd\', \'expanded_url\': \'https://www.linkedin.com/company/the-erdős-institute\', \'display_url\': \'linkedin.com/company/the-er…\', \'indices\': [57, 80]}]}}, \'protected\': False, \'followers_count\': 304, \'friends_count\': 228, \'listed_count\': 4, \'created_at\': \'Sun Mar 12 00:52:05 +0000 2017\', \'favourites_count\': 708, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 516, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/840727059001417728/1610744135\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': False, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, id=840727059001417728, id_str=\'840727059001417728\', name=\'The Erdős Institute\', screen_name=\'erdosinstitute\', location=\'United States\', description=\'Helping PhDs get jobs they love. \\nFollow us on LinkedIn: https://t.co/pNGRyjotVd\', url=\'https://t.co/MoNoEsytrh\', entities={\'url\': {\'urls\': [{\'url\': \'https://t.co/MoNoEsytrh\', \'expanded_url\': \'http://erdosinstitute.org\', \'display_url\': \'erdosinstitute.org\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': [{\'url\': \'https://t.co/pNGRyjotVd\', \'expanded_url\': \'https://www.linkedin.com/company/the-erdős-institute\', \'display_url\': \'linkedin.com/company/the-er…\', \'indices\': [57, 80]}]}}, protected=False, followers_count=304, friends_count=228, listed_count=4, created_at=datetime.datetime(2017, 3, 12, 0, 52, 5), favourites_count=708, utc_offset=None, time_zone=None, geo_enabled=True, verified=False, statuses_count=516, lang=None, contributors_enabled=False, is_translator=False, is_translation_enabled=False, profile_background_color=\'F5F8FA\', profile_background_image_url=None, profile_background_image_url_https=None, profile_background_tile=False, profile_image_url=\'http://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', profile_image_url_https=\'https://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', profile_banner_url=\'https://pbs.twimg.com/profile_banners/840727059001417728/1610744135\', profile_link_color=\'1DA1F2\', profile_sidebar_border_color=\'C0DEED\', profile_sidebar_fill_color=\'DDEEF6\', profile_text_color=\'333333\', profile_use_background_image=True, has_extended_profile=False, default_profile=True, default_profile_image=False, following=False, follow_request_sent=False, notifications=False, translator_type=\'none\', withheld_in_countries=[]), user=User(_api=<tweepy.api.API object at 0x00000208D4918948>, _json={\'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'name\': \'The Erdős Institute\', \'screen_name\': \'erdosinstitute\', \'location\': \'United States\', \'description\': \'Helping PhDs get jobs they love. \\nFollow us on LinkedIn: https://t.co/pNGRyjotVd\', \'url\': \'https://t.co/MoNoEsytrh\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/MoNoEsytrh\', \'expanded_url\': \'http://erdosinstitute.org\', \'display_url\': \'erdosinstitute.org\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': [{\'url\': \'https://t.co/pNGRyjotVd\', \'expanded_url\': \'https://www.linkedin.com/company/the-erdős-institute\', \'display_url\': \'linkedin.com/company/the-er…\', \'indices\': [57, 80]}]}}, \'protected\': False, \'followers_count\': 304, \'friends_count\': 228, \'listed_count\': 4, \'created_at\': \'Sun Mar 12 00:52:05 +0000 2017\', \'favourites_count\': 708, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 516, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/840727059001417728/1610744135\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': False, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, id=840727059001417728, id_str=\'840727059001417728\', name=\'The Erdős Institute\', screen_name=\'erdosinstitute\', location=\'United States\', description=\'Helping PhDs get jobs they love. \\nFollow us on LinkedIn: https://t.co/pNGRyjotVd\', url=\'https://t.co/MoNoEsytrh\', entities={\'url\': {\'urls\': [{\'url\': \'https://t.co/MoNoEsytrh\', \'expanded_url\': \'http://erdosinstitute.org\', \'display_url\': \'erdosinstitute.org\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': [{\'url\': \'https://t.co/pNGRyjotVd\', \'expanded_url\': \'https://www.linkedin.com/company/the-erdős-institute\', \'display_url\': \'linkedin.com/company/the-er…\', \'indices\': [57, 80]}]}}, protected=False, followers_count=304, friends_count=228, listed_count=4, created_at=datetime.datetime(2017, 3, 12, 0, 52, 5), favourites_count=708, utc_offset=None, time_zone=None, geo_enabled=True, verified=False, statuses_count=516, lang=None, contributors_enabled=False, is_translator=False, is_translation_enabled=False, profile_background_color=\'F5F8FA\', profile_background_image_url=None, profile_background_image_url_https=None, profile_background_tile=False, profile_image_url=\'http://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', profile_image_url_https=\'https://pbs.twimg.com/profile_images/840727791641452544/9dkgoMdN_normal.jpg\', profile_banner_url=\'https://pbs.twimg.com/profile_banners/840727059001417728/1610744135\', profile_link_color=\'1DA1F2\', profile_sidebar_border_color=\'C0DEED\', profile_sidebar_fill_color=\'DDEEF6\', profile_text_color=\'333333\', profile_use_background_image=True, has_extended_profile=False, default_profile=True, default_profile_image=False, following=False, follow_request_sent=False, notifications=False, translator_type=\'none\', withheld_in_countries=[]), geo=None, coordinates=None, place=None, contributors=None, retweeted_status=Status(_api=<tweepy.api.API object at 0x00000208D4918948>, _json={\'created_at\': \'Mon May 17 14:56:49 +0000 2021\', \'id\': 1394305912622489602, \'id_str\': \'1394305912622489602\', \'text\': \'@RacineSwick @WorkWithVari This was during one of this week’s @erdosinstitute data science lectures!\', \'truncated\': False, \'entities\': {\'hashtags\': [], \'symbols\': [], \'user_mentions\': [{\'screen_name\': \'RacineSwick\', \'name\': \'Racine Swick\', \'id\': 1231793881672486912, \'id_str\': \'1231793881672486912\', \'indices\': [0, 12]}, {\'screen_name\': \'WorkWithVari\', \'name\': \'Vari®\', \'id\': 1266638119, \'id_str\': \'1266638119\', \'indices\': [13, 26]}, {\'screen_name\': \'erdosinstitute\', \'name\': \'The Erdős Institute\', \'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'indices\': [62, 77]}], \'urls\': []}, \'source\': \'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>\', \'in_reply_to_status_id\': 1394294574806278144, \'in_reply_to_status_id_str\': \'1394294574806278144\', \'in_reply_to_user_id\': 1231793881672486912, \'in_reply_to_user_id_str\': \'1231793881672486912\', \'in_reply_to_screen_name\': \'RacineSwick\', \'user\': {\'id\': 913058608883142657, \'id_str\': \'913058608883142657\', \'name\': \'Patrick Vallely 🛰️🌌🔭\', \'screen_name\': \'AstronomerPat\', \'location\': \'NV➡OK➡OH\', \'description\': \'Studying supernovae with ASAS-SN & TESS | @NSFGRFP fellow at The™ @OhioState | @UofOklahoma Class of 2017 | Amateur photographer and big cat enthusiast | He/Him\', \'url\': \'https://t.co/AEt85T0LIF\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/AEt85T0LIF\', \'expanded_url\': \'https://u.osu.edu/vallely.7/\', \'display_url\': \'u.osu.edu/vallely.7/\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': []}}, \'protected\': False, \'followers_count\': 1262, \'friends_count\': 664, \'listed_count\': 12, \'created_at\': \'Wed Sep 27 15:11:50 +0000 2017\', \'favourites_count\': 39911, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 6678, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/913058608883142657/1540879883\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': True, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, \'geo\': None, \'coordinates\': None, \'place\': None, \'contributors\': None, \'is_quote_status\': False, \'retweet_count\': 1, \'favorite_count\': 3, \'favorited\': False, \'retweeted\': False, \'lang\': \'en\'}, created_at=datetime.datetime(2021, 5, 17, 14, 56, 49), id=1394305912622489602, id_str=\'1394305912622489602\', text=\'@RacineSwick @WorkWithVari This was during one of this week’s @erdosinstitute data science lectures!\', truncated=False, entities={\'hashtags\': [], \'symbols\': [], \'user_mentions\': [{\'screen_name\': \'RacineSwick\', \'name\': \'Racine Swick\', \'id\': 1231793881672486912, \'id_str\': \'1231793881672486912\', \'indices\': [0, 12]}, {\'screen_name\': \'WorkWithVari\', \'name\': \'Vari®\', \'id\': 1266638119, \'id_str\': \'1266638119\', \'indices\': [13, 26]}, {\'screen_name\': \'erdosinstitute\', \'name\': \'The Erdős Institute\', \'id\': 840727059001417728, \'id_str\': \'840727059001417728\', \'indices\': [62, 77]}], \'urls\': []}, source=\'Twitter for iPhone\', source_url=\'http://twitter.com/download/iphone\', in_reply_to_status_id=1394294574806278144, in_reply_to_status_id_str=\'1394294574806278144\', in_reply_to_user_id=1231793881672486912, in_reply_to_user_id_str=\'1231793881672486912\', in_reply_to_screen_name=\'RacineSwick\', author=User(_api=<tweepy.api.API object at 0x00000208D4918948>, _json={\'id\': 913058608883142657, \'id_str\': \'913058608883142657\', \'name\': \'Patrick Vallely 🛰️🌌🔭\', \'screen_name\': \'AstronomerPat\', \'location\': \'NV➡OK➡OH\', \'description\': \'Studying supernovae with ASAS-SN & TESS | @NSFGRFP fellow at The™ @OhioState | @UofOklahoma Class of 2017 | Amateur photographer and big cat enthusiast | He/Him\', \'url\': \'https://t.co/AEt85T0LIF\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/AEt85T0LIF\', \'expanded_url\': \'https://u.osu.edu/vallely.7/\', \'display_url\': \'u.osu.edu/vallely.7/\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': []}}, \'protected\': False, \'followers_count\': 1262, \'friends_count\': 664, \'listed_count\': 12, \'created_at\': \'Wed Sep 27 15:11:50 +0000 2017\', \'favourites_count\': 39911, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 6678, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/913058608883142657/1540879883\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': True, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, id=913058608883142657, id_str=\'913058608883142657\', name=\'Patrick Vallely 🛰️🌌🔭\', screen_name=\'AstronomerPat\', location=\'NV➡OK➡OH\', description=\'Studying supernovae with ASAS-SN & TESS | @NSFGRFP fellow at The™ @OhioState | @UofOklahoma Class of 2017 | Amateur photographer and big cat enthusiast | He/Him\', url=\'https://t.co/AEt85T0LIF\', entities={\'url\': {\'urls\': [{\'url\': \'https://t.co/AEt85T0LIF\', \'expanded_url\': \'https://u.osu.edu/vallely.7/\', \'display_url\': \'u.osu.edu/vallely.7/\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': []}}, protected=False, followers_count=1262, friends_count=664, listed_count=12, created_at=datetime.datetime(2017, 9, 27, 15, 11, 50), favourites_count=39911, utc_offset=None, time_zone=None, geo_enabled=True, verified=False, statuses_count=6678, lang=None, contributors_enabled=False, is_translator=False, is_translation_enabled=False, profile_background_color=\'F5F8FA\', profile_background_image_url=None, profile_background_image_url_https=None, profile_background_tile=False, profile_image_url=\'http://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', profile_image_url_https=\'https://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', profile_banner_url=\'https://pbs.twimg.com/profile_banners/913058608883142657/1540879883\', profile_link_color=\'1DA1F2\', profile_sidebar_border_color=\'C0DEED\', profile_sidebar_fill_color=\'DDEEF6\', profile_text_color=\'333333\', profile_use_background_image=True, has_extended_profile=True, default_profile=True, default_profile_image=False, following=False, follow_request_sent=False, notifications=False, translator_type=\'none\', withheld_in_countries=[]), user=User(_api=<tweepy.api.API object at 0x00000208D4918948>, _json={\'id\': 913058608883142657, \'id_str\': \'913058608883142657\', \'name\': \'Patrick Vallely 🛰️🌌🔭\', \'screen_name\': \'AstronomerPat\', \'location\': \'NV➡OK➡OH\', \'description\': \'Studying supernovae with ASAS-SN & TESS | @NSFGRFP fellow at The™ @OhioState | @UofOklahoma Class of 2017 | Amateur photographer and big cat enthusiast | He/Him\', \'url\': \'https://t.co/AEt85T0LIF\', \'entities\': {\'url\': {\'urls\': [{\'url\': \'https://t.co/AEt85T0LIF\', \'expanded_url\': \'https://u.osu.edu/vallely.7/\', \'display_url\': \'u.osu.edu/vallely.7/\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': []}}, \'protected\': False, \'followers_count\': 1262, \'friends_count\': 664, \'listed_count\': 12, \'created_at\': \'Wed Sep 27 15:11:50 +0000 2017\', \'favourites_count\': 39911, \'utc_offset\': None, \'time_zone\': None, \'geo_enabled\': True, \'verified\': False, \'statuses_count\': 6678, \'lang\': None, \'contributors_enabled\': False, \'is_translator\': False, \'is_translation_enabled\': False, \'profile_background_color\': \'F5F8FA\', \'profile_background_image_url\': None, \'profile_background_image_url_https\': None, \'profile_background_tile\': False, \'profile_image_url\': \'http://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_image_url_https\': \'https://pbs.twimg.com/profile_images/1248636399022211073/sNZtQrfX_normal.jpg\', \'profile_banner_url\': \'https://pbs.twimg.com/profile_banners/913058608883142657/1540879883\', \'profile_link_color\': \'1DA1F2\', \'profile_sidebar_border_color\': \'C0DEED\', \'profile_sidebar_fill_color\': \'DDEEF6\', \'profile_text_color\': \'333333\', \'profile_use_background_image\': True, \'has_extended_profile\': True, \'default_profile\': True, \'default_profile_image\': False, \'following\': False, \'follow_request_sent\': False, \'notifications\': False, \'translator_type\': \'none\', \'withheld_in_countries\': []}, id=913058608883142657, id_str=\'913058608883142657\', name=\'Patrick Vallely 🛰️🌌🔭\', screen_name=\'AstronomerPat\', location=\'NV➡OK➡OH\', description=\'Studying supernovae with ASAS-SN & TESS | @NSFGRFP fellow at The™ @OhioState | @UofOklahoma Class of 2017 | Amateur photographer and big cat enthusiast | He/Him\', url=\'https://t.co/AEt85T0LIF\', entities={\'url\': {\'urls\': [{\'url\': \'https://t.co/AEt85T0LIF\', \'expanded_url\': \'https://u.osu.edu/vallely.7/\', \'display_url\': \'u.osu.edu/vallely.7/\', \'indices\': [0, 23]}]}, \'description\': {\'urls\': []}}'''

In [254]:
tweet.text

'RT @AstronomerPat: @RacineSwick @WorkWithVari This was during one of this week’s @erdosinstitute data science lectures!'